# Topic Analysis

In [3]:
import datetime

import pandas as pd
import spacy
import re
import string
import numpy as np

import seaborn as sns 
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

import nltk
from nltk.corpus import stopwords
from spacy.tokens import Token
from tqdm import tqdm


import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import pyLDAvis.gensim
pyLDAvis.enable_notebook()
from ipywidgets import interact

%matplotlib inline
np.random.seed(500)

In [4]:
df = pd.read_csv("..//data//Womens Clothing E-Commerce Reviews Sentiment v2.csv")

In [5]:
## Discarding a empty reviews records

df=df[((df.processed_Review_text.isna()==False) & (df.processed_Review_text.isnull()==False) & (df.processed_Review_text!=""))]

## Tokenizing the review text

In [6]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [7]:
def buildLDA(processed_docs, ip_num_topics=4):
    dictionary = gensim.corpora.Dictionary(processed_docs)

    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]

    lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=ip_num_topics, id2word=dictionary, passes=2, workers=4)
    
    return dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf

In [8]:
def LDAtopicSummary(model, displaysubplot=[2,2]):
    for idx, topic in model.print_topics(-1):
        print('Topic: {} Word: {}'.format(idx, topic))

In [9]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        #print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords',"Text"]
    return(sent_topics_df)


## LDA - Non-Recommendation 

### Non-Recommendation & Negative reviews

In [10]:
Neg_NR_processed_docs=df["processed_Review_text"][(df["Recommended IND"]==0 ) & (df["PA_Polarity"]<0)].map(preprocess)
dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf =buildLDA(Neg_NR_processed_docs,8)
len(Neg_NR_processed_docs)

893

In [11]:
#LDAtopicSummary(lda_model_tfidf, [2,4])
pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, dictionary=lda_model_tfidf.id2word)

C:\Users\Dell\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.014980  0.026279       1        1  28.490601
3      0.028624  0.051967       2        1  14.955489
4     -0.027495 -0.001457       3        1  12.896567
5     -0.015787 -0.007342       4        1  11.370795
0     -0.059719  0.026126       5        1   9.719639
6     -0.019040 -0.061261       6        1   9.616147
1      0.020737 -0.009106       7        1   8.159925
7      0.057701 -0.025208       8        1   4.790836, topic_info=    Category        Freq            Term       Total  loglift  logprob
110  Default  132.000000           shirt  132.000000  30.0000  30.0000
69   Default  107.000000         sweater  107.000000  29.0000  29.0000
45   Default   56.000000            poor   56.000000  28.0000  28.0000
47   Default  108.000000         quality  108.000000  27.0000  27.0000
114  Default   45.000000          arrive   45.000000  26.0000  26.0000
57   Default   88.000000            wash   88.000000  25.0000  25.0000
176  Default   31.000000       shapeless   31.000000  24.0000  24.0000
146  Default   50.000000            hole   50.000000  23.0000  23.0000
158  Default   56.000000          blouse   56.000000  22.0000  22.0000
107  Default   83.000000            wide   83.000000  21.0000  21.0000
4    Default  143.000000          fabric  143.000000  20.0000  20.0000
41   Default  141.000000            wear  141.000000  19.0000  19.0000
46   Default   62.000000           price   62.000000  18.0000  18.0000
170  Default   23.000000          middle   23.000000  17.0000  17.0000
203  Default   33.000000          jacket   33.000000  16.0000  16.0000
2    Default  112.000000    disappointed  112.000000  15.0000  15.0000
87   Default   70.000000           skirt   70.000000  14.0000  14.0000
144  Default   43.000000         stretch   43.000000  13.0000  13.0000
101  Default   75.000000          petite   75.000000  12.0000  12.0000
139  Default   31.000000            lace   31.000000  11.0000  11.0000
130  Default   35.000000            suit   35.000000  10.0000  10.0000
88   Default   30.000000            hard   30.000000   9.0000   9.0000
55   Default   51.000000            send   51.000000   8.0000   8.0000
181  Default   45.000000            seam   45.000000   7.0000   7.0000
174  Default   40.000000           wrong   40.000000   6.0000   6.0000
200  Default   31.000000           bulky   31.000000   5.0000   5.0000
82   Default   79.000000          sleeve   79.000000   4.0000   4.0000
5    Default   99.000000            feel   99.000000   3.0000   3.0000
52   Default   34.000000            item   34.000000   2.0000   2.0000
178  Default   31.000000            pair   31.000000   1.0000   1.0000
..       ...         ...             ...         ...      ...      ...
188   Topic8    4.273527           extra   39.598122   0.8121  -4.8448
165   Topic8    2.509382           curvy   23.610430   0.7968  -5.3772
79    Topic8    3.384044        probably   31.905451   0.7948  -5.0782
126   Topic8    4.054955            fall   38.424206   0.7897  -4.8973
1     Topic8    5.339257           cheap   50.804893   0.7856  -4.6222
34    Topic8    5.161794            hope   49.495556   0.7779  -4.6560
66    Topic8    4.064749         problem   39.571384   0.7627  -4.8949
115   Topic8    4.140192            base   40.795254   0.7506  -4.8765
85    Topic8    4.351517  disappointment   44.394772   0.7159  -4.8267
87    Topic8    6.477071           skirt   70.921143   0.6452  -4.4290
2     Topic8    9.464507    disappointed  112.054199   0.5670  -4.0497
9     Topic8    6.967616          pretty   79.675728   0.6018  -4.3560
109   Topic8    5.908388            hang   67.041473   0.6095  -4.5209
133   Topic8    4.968831      definitely   52.955482   0.6722  -4.6941
82    Topic8    6.409880          sleeve   79.406975   0.5217  -4.4394
135   Topic8    5.193485    unflattering   58.102413   0.6237  -4.6499
110   Top

### Non-Recommendation & Positive reviews

In [12]:
Pos_NR_processed_docs=df["processed_Review_text"][(df["Recommended IND"]==0 ) & (df["PA_Polarity"]>0)].map(preprocess)
dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf =buildLDA(Pos_NR_processed_docs,9)

In [13]:
# LDAtopicSummary(lda_model_tfidf, [2,4])
pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, dictionary=lda_model_tfidf.id2word)

C:\Users\Dell\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.007881 -0.010540       1        1  19.068010
3      0.000792  0.033774       2        1  14.125826
1      0.004968 -0.010619       3        1  12.687349
4     -0.002397  0.007174       4        1  12.506589
7      0.027136  0.001173       5        1  11.123368
2      0.004206 -0.018516       6        1  11.024632
5      0.034017 -0.005046       7        1   7.542139
0     -0.046397 -0.026094       8        1   6.829833
8     -0.030205  0.028695       9        1   5.092258, topic_info=    Category        Freq        Term       Total  loglift  logprob
255  Default  185.000000        wash  185.000000  30.0000  30.0000
423  Default   71.000000      shrink   71.000000  29.0000  29.0000
289  Default  157.000000      blouse  157.000000  28.0000  28.0000
381  Default  227.000000        pant  227.000000  27.0000  27.0000
230  Default   91.000000       torso   91.000000  26.0000  26.0000
277  Default  114.000000        lace  114.000000  25.0000  25.0000
514  Default   80.000000       bulky   80.000000  24.0000  24.0000
98   Default   89.000000        tank   89.000000  23.0000  23.0000
358  Default  101.000000        knee  101.000000  22.0000  22.0000
48   Default  586.000000        look  586.000000  21.0000  21.0000
40   Default  446.000000        wear  446.000000  20.0000  20.0000
319  Default  320.000000        nice  320.000000  19.0000  19.0000
234  Default  125.000000      figure  125.000000  18.0000  18.0000
323  Default   72.000000        poor   72.000000  17.0000  17.0000
332  Default   33.000000        life   33.000000  16.0000  16.0000
0    Default  209.000000       cheap  209.000000  15.0000  15.0000
60   Default  368.000000       short  368.000000  14.0000  14.0000
124  Default   70.000000        cami   70.000000  13.0000  13.0000
96   Default  315.000000        good  315.000000  12.0000  12.0000
65   Default   82.000000  underneath   82.000000  11.0000  11.0000
77   Default  109.000000       itchy  109.000000  10.0000  10.0000
49   Default  395.000000    material  395.000000   9.0000   9.0000
153  Default  428.000000       shirt  428.000000   8.0000   8.0000
108  Default  261.000000      online  261.000000   7.0000   7.0000
30   Default  254.000000        long  254.000000   6.0000   6.0000
120  Default  100.000000     strange  100.000000   5.0000   5.0000
83   Default  195.000000       super  195.000000   4.0000   4.0000
333  Default   50.000000        real   50.000000   3.0000   3.0000
182  Default   92.000000        suit   92.000000   2.0000   2.0000
62   Default  341.000000     sweater  341.000000   1.0000   1.0000
..       ...         ...         ...         ...      ...      ...
30    Topic9   24.456396        long  254.949432   0.6333  -4.7419
417   Topic9    8.462843    previous   61.466454   0.9946  -5.8032
120   Topic9   12.066960     strange  100.506981   0.8577  -5.4484
96    Topic9   25.391121        good  315.959167   0.4562  -4.7044
40    Topic9   31.251007        wear  446.197998   0.3187  -4.4968
77    Topic9   12.327802       itchy  109.379921   0.7945  -5.4270
62    Topic9   22.732313     sweater  341.338562   0.2684  -4.8150
48    Topic9   30.118586        look  586.407837   0.0086  -4.5337
60    Topic9   22.172804       short  368.328552   0.1673  -4.8400
319   Topic9   20.384991        nice  320.888031   0.2212  -4.9240
61    Topic9   17.584721      sleeve  246.976959   0.3352  -5.0718
35    Topic9   22.883520      return  448.144348   0.0028  -4.8084
7     Topic9   16.215696        high  225.525635   0.3450  -5.1529
114   Topic9   22.768137      fabric  498.347137  -0.1085  -4.8135
15    Topic9   23.444757        size  537.140991  -0.1542  -4.7842
47    Topic9   23.584530        like  545.347473  -0.1634  -4.7782
25    Topic9   21.090563       color  437.859009  -0.0556  -4.8900
16    Topic9   20.864292       small  438.426849  -0.0677  -4.9008
87    Topic9 

In [14]:
def demo(title):
    return([" ".join(i) for i in df_dominant_topic["Text"][df_dominant_topic["Dominant_Topic"]==title].tolist()][:20])

df_dominant_topic = format_topics_sentences(lda_model_tfidf,bow_corpus,list(Pos_NR_processed_docs) )
interact(demo, title=range(9));

interactive(children=(Dropdown(description='title', options=(0, 1, 2, 3, 4, 5, 6, 7, 8), value=0), Output()), …

## LDA - Recommended Data

### Recommendation & Negative reviews

In [15]:
Neg_R_processed_docs=df["processed_Review_text"][(df["Recommended IND"]==1 ) & (df["PA_Polarity"]<0)].map(preprocess)
dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf =buildLDA(Neg_R_processed_docs,10)
len(Neg_R_processed_docs)

817

In [16]:
# LDAtopicSummary(lda_model_tfidf, [2,4])
pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, dictionary=lda_model_tfidf.id2word)

C:\Users\Dell\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.014708 -0.027572       1        1  30.036364
9     -0.032647 -0.033039       2        1  16.357172
3      0.000156 -0.031070       3        1  12.908723
7     -0.000458  0.053335       4        1   7.640260
4     -0.029894  0.040851       5        1   7.012371
6     -0.042751 -0.019843       6        1   6.954311
1      0.045776 -0.031214       7        1   5.446648
0      0.010954  0.043640       8        1   4.702253
5     -0.012742  0.005682       9        1   4.491369
8      0.076315 -0.000770      10        1   4.450534, topic_info=    Category        Freq        Term       Total  loglift  logprob
136  Default  102.000000      casual  102.000000  30.0000  30.0000
1    Default  218.000000       dress  218.000000  29.0000  29.0000
125  Default   70.000000     sweater   70.000000  28.0000  28.0000
35   Default   49.000000       price   49.000000  27.0000  27.0000
29   Default   90.000000       black   90.000000  26.0000  26.0000
127  Default   60.000000        sale   60.000000  25.0000  25.0000
178  Default   42.000000      blouse   42.000000  24.0000  24.0000
109  Default   79.000000        jean   79.000000  23.0000  23.0000
51   Default   60.000000      design   60.000000  22.0000  22.0000
63   Default  151.000000        look  151.000000  21.0000  21.0000
38   Default  166.000000        wear  166.000000  20.0000  20.0000
55   Default   62.000000       loose   62.000000  19.0000  19.0000
159  Default   60.000000        wish   60.000000  18.0000  18.0000
95   Default   72.000000      pretty   72.000000  17.0000  17.0000
4    Default  126.000000        love  126.000000  16.0000  16.0000
164  Default   45.000000      pocket   45.000000  15.0000  15.0000
31   Default   35.000000      dressy   35.000000  14.0000  14.0000
106  Default   77.000000        soft   77.000000  13.0000  13.0000
163  Default   37.000000        vest   37.000000  12.0000  12.0000
129  Default   59.000000       comfy   59.000000  11.0000  11.0000
76   Default   68.000000    purchase   68.000000  10.0000  10.0000
79   Default   72.000000      sleeve   72.000000   9.0000   9.0000
62   Default   60.000000       green   60.000000   8.0000   8.0000
2    Default   67.000000  flattering   67.000000   7.0000   7.0000
50   Default  130.000000       color  130.000000   6.0000   6.0000
134  Default   72.000000       white   72.000000   5.0000   5.0000
23   Default   83.000000       skirt   83.000000   4.0000   4.0000
145  Default   66.000000     flatter   66.000000   3.0000   3.0000
18   Default   73.000000      length   73.000000   2.0000   2.0000
45   Default   85.000000        need   85.000000   1.0000   1.0000
..       ...         ...         ...         ...      ...      ...
133  Topic10    4.480764        navy   39.301529   0.9407  -4.7480
22   Topic10    5.578410     quality   49.649883   0.9261  -4.5289
23   Topic10    9.224848       skirt   83.533813   0.9088  -4.0259
48   Topic10    6.770247        wash   62.491253   0.8897  -4.3353
183  Topic10    2.336592   versatile   21.660608   0.8853  -5.3991
199  Topic10    3.398463      lovely   31.673782   0.8800  -5.0245
166  Topic10    4.578743        boxy   42.807194   0.8769  -4.7264
29   Topic10    8.935792       black   90.164597   0.8006  -4.0578
128  Topic10    2.638466    cardigan   27.678375   0.7617  -5.2776
159  Topic10    5.749031        wish   60.340794   0.7612  -4.4988
45   Topic10    7.995866        need   85.199776   0.7461  -4.1689
0    Topic10    5.960535        bust   65.490280   0.7154  -4.4627
76   Topic10    6.220349    purchase   68.864754   0.7078  -4.4200
71   Topic10    2.513448       drape   27.875296   0.7061  -5.3262
18   Topic10    6.469005      length   73.920212   0.6762  -4.3808
131  Topic10    2.906633       strap   33.895897   0.6558  -5.1808
175  Topic10    3.736664       issue   44.317867   0.6390  -4.9296
78   Topic10    6.567169  

In [17]:
df_dominant_topic = format_topics_sentences(lda_model_tfidf,bow_corpus,list(Neg_R_processed_docs) )
def demo(title):
    return([" ".join(i) for i in df_dominant_topic["Text"][df_dominant_topic["Dominant_Topic"]==title].tolist()][:20])

interact(demo, title=range(10));

interactive(children=(Dropdown(description='title', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0), Output()…

### Recommendation & Positive reviews

In [18]:
Pos_R_processed_docs=df["processed_Review_text"][(df["Recommended IND"]==1 ) & (df["PA_Polarity"]>0)].map(preprocess)
dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf =buildLDA(Pos_R_processed_docs,12)
len(Pos_R_processed_docs)

17609

In [19]:
pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, dictionary=lda_model_tfidf.id2word)

C:\Users\Dell\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.055468 -0.007786       1        1  14.933364
9     -0.010207 -0.002405       2        1  11.024078
10    -0.034924 -0.007952       3        1   9.260863
0     -0.007791  0.005671       4        1   8.528136
6      0.016882  0.006730       5        1   8.522532
5     -0.025901  0.006100       6        1   7.606385
8     -0.011088 -0.003511       7        1   7.414196
4      0.007737 -0.031258       8        1   6.857050
11     0.002533 -0.027969       9        1   6.753058
2     -0.001498 -0.003347      10        1   6.547426
3      0.008171  0.059088      11        1   6.546831
7      0.000618  0.006639      12        1   6.006084, topic_info=     Category         Freq         Term        Total  loglift  logprob
430   Default   447.000000      stylish   447.000000  30.0000  30.0000
409   Default   230.000000     everyday   230.000000  29.0000  29.0000
22    Default  1133.000000   compliment  1133.000000  28.0000  28.0000
1360  Default   178.000000       kimono   178.000000  27.0000  27.0000
204   Default   953.000000       jacket   953.000000  26.0000  26.0000
1032  Default   308.000000         suit   308.000000  25.0000  25.0000
232   Default   782.000000         tank   782.000000  24.0000  24.0000
1543  Default    88.000000         sock    88.000000  23.0000  23.0000
211   Default  1796.000000      sweater  1796.000000  22.0000  22.0000
291   Default   745.000000      receive   745.000000  21.0000  21.0000
26    Default   377.000000     jumpsuit   377.000000  20.0000  20.0000
729   Default   123.000000        chino   123.000000  19.0000  19.0000
35    Default  1924.000000        shirt  1924.000000  18.0000  18.0000
32    Default   962.000000      legging   962.000000  17.0000  17.0000
20    Default  1411.000000         true  1411.000000  16.0000  16.0000
255   Default   897.000000     shoulder   897.000000  15.0000  15.0000
149   Default   368.000000         cozy   368.000000  14.0000  14.0000
536   Default   250.000000  interesting   250.000000  13.0000  13.0000
724   Default   290.000000     addition   290.000000  12.0000  12.0000
6     Default  3835.000000        dress  3835.000000  11.0000  11.0000
1035  Default    87.000000       eyelet    87.000000  10.0000  10.0000
819   Default   157.000000    yesterday   157.000000   9.0000   9.0000
938   Default   157.000000     layering   157.000000   8.0000   8.0000
201   Default  1118.000000        comfy  1118.000000   7.0000   7.0000
812   Default   172.000000      washing   172.000000   6.0000   6.0000
776   Default   282.000000     daughter   282.000000   5.0000   5.0000
267   Default   184.000000       poncho   184.000000   4.0000   4.0000
257   Default   502.000000   embroidery   502.000000   3.0000   3.0000
1     Default  2291.000000  comfortable  2291.000000   2.0000   2.0000
460   Default   145.000000         life   145.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
291   Topic12   120.960976      receive   745.812134   0.9934  -5.1388
211   Topic12   202.342392      sweater  1796.150757   0.6290  -4.6244
22    Topic12   144.390762   compliment  1133.709473   0.7517  -4.9618
6     Topic12   243.492950        dress  3835.822021   0.0553  -4.4392
178   Topic12   156.910980         cute  1960.529175   0.2871  -4.8786
28    Topic12   181.291626         wear  2800.631836   0.0749  -4.7342
12    Topic12   188.936295         love  3161.760498  -0.0051  -4.6929
111   Topic12    96.720802        happy   884.561707   0.5991  -5.3625
43    Topic12   163.593231         look  2569.732422   0.0582  -4.8369
197   Topic12   134.241013    beautiful  1836.145996   0.1966  -5.0347
259   Topic12    94.205582       lovely   912.646729   0.5415  -5.3888
48    Topic12   170.042877         size  3075.420166  -0.0827  -4.7983
15    Topic12   141.124680        order  2192.695312   0.0692  -4.9847
39    

In [20]:
# LDAtopicSummary(lda_model_tfidf, [2,4])
df_dominant_topic = format_topics_sentences(lda_model_tfidf,bow_corpus,list(Pos_R_processed_docs) )
def demo(title):
    return([" ".join(i) for i in df_dominant_topic["Text"][df_dominant_topic["Dominant_Topic"]==title].tolist()][:20])

interact(demo, title=range(12));

interactive(children=(Dropdown(description='title', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), value=0), …